In [1]:
import numpy as np
import healpy as hp
import pandas as pd
from astropy.cosmology import Planck18 as cosmo
import os
output_dir = "mask1_err"
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

###################################################################################

def superpixel(subpix, nside_subpix, nside_superpix):
    """
    Return the indices of the super-pixels which contain each of the sub-pixels.
    """
    if nside_subpix == nside_superpix:
        return subpix
    theta, phi = hp.pix2ang(nside_subpix, subpix)
    return hp.ang2pix(nside_superpix, theta, phi)



def subpixel(superpix, nside_superpix, nside_subpix):
    """
    Return the indices of sub-pixels (resolution nside_subpix) within
    the super-pixel with (resolution nside_superpix).
    """
    if nside_superpix == nside_subpix:
        return superpix
    vec = hp.pix2vec(nside_superpix, superpix)
    radius = np.degrees(2. * hp.max_pixrad(nside_superpix))
    subpix = query_disc(nside_subpix, vec, radius=radius)
    pix_for_subpix = superpixel(subpix, nside_subpix, nside_superpix)
    # Might be able to speed up array indexing...
    return subpix[pix_for_subpix == superpix]

######################################################################################

for k in range(3):
    f = '../../../../data/quasar_sample_' + str(k+1) + '.csv'
    df0 = pd.read_csv(f)

    fd1 = df0[(df0.b>=40)]
    fd2 = df0[(df0.b<=0)]
    fd3 = fd2[(fd2.b<=-60)]

    fd4 = pd.concat([fd1, fd3])
    fd4.reset_index(inplace = True, drop = True)
    r = fd4['r'].to_numpy()    
    th = fd4['theta'].to_numpy()
    ph = fd4['phi'].to_numpy()
    l = fd4['l'].to_numpy()
    b = fd4['b'].to_numpy()
    z = fd4['redshift'].to_numpy()
    
    print(np.max(z), np.min(z))

#########################################################  
    sigma = np.zeros(len(z)) 
    err_z = np.zeros(len(z))
    for ii in range(len(z)):
        sigma[ii] = 0.10*z[ii]         # adding 10 % error 
        lower_bound = z[ii] - sigma[ii] 
        upper_bound = z[ii] + sigma[ii]
        while True: 
            err_z[ii] = np.random.normal(z[ii], sigma[ii]) 
            if lower_bound <= err_z[ii] <= upper_bound: 
                break
        
    err_dist = cosmo.comoving_distance(err_z).value
    print(np.max(err_z), np.min(err_z))
#########################################################
    
    Nside=64                   # nside of the subpixel
    Npix=hp.nside2npix(Nside)

    nside = 8                  # nside of the superpixel
    super_pix = hp.nside2npix(nside)   

    m=np.zeros(Npix, dtype= int)
    for i in range(len(r)):
        px=hp.ang2pix(Nside, th[i], ph[i])
        m [px]+=1

    sp=np.zeros(Npix)
    for j in range(Npix):
        sp[j]= superpixel(j, Nside, nside)               

    # Stack the arrays column-wise
    data1 = np.column_stack((sp,m))
    df = pd.DataFrame(data1, columns=['spp', 'mm'])
    P=[]
    for j in range(super_pix):
        fd = df[df.spp == j]
        fd.reset_index(inplace = True, drop = True)
        f = (len(fd[fd.mm!=0]))/len(fd)
        if(f>=0.90):
            P.append(j)

    p = np.array(P)
    df1 = pd.DataFrame(data=p)
    f1 = 'mask1_err/non_zero_pix_id_' + str(k+1) + '.dat'

    df1.to_csv(f1,sep='\t',header = None,index = False)
    
        
    op = 'mask1_err/masked_sample_' + str(k+1) + '.dat'
    f2=open(op, 'w')
    for i in range(len(r)):
        px=hp.ang2pix(8, th[i], ph[i])
        for j in range(len(p)):
            if(px==p[j]):
                f2.write("%f\t%f\t%f\n"%(err_dist[i],th[i],ph[i]))

    f2.close()  


0.9999972581863404 0.36
1.09766753162711 0.32569509770419
2.199995517730713 1.0
2.416460883442145 0.9005166801227915
2.7999932765960693 2.2
3.0729353285442333 1.9863522758946819
